# Example of CESM2-MARBL Flows

## Imports
We start first with importing the "ingredients" of `esds-funnel`, including:
* [xpersist](https://xpersist.readthedocs.io/en/latest/)
* [xcollection](https://xcollection.readthedocs.io/en/latest/)
* [Prefect](https://www.prefect.io/)
* [intake-esm](https://intake-esm.readthedocs.io/en/stable/)

In [1]:
import os

os.environ["PREFECT__FLOWS__CHECKPOINTING"] = "True"

import tempfile
import time

import numpy as np
import xarray as xr
import xcollection as xc
from distributed import Client
import intake
from ncar_jobqueue import NCARCluster
from prefect import Flow, Parameter, task
from xpersist import CacheStore, XpersistResult
import yaml

/glade/u/home/mgrover/.local/lib/python3.9/site-packages/dask_jobqueue/core.py:19: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/glade/u/home/mgrover/.local/lib/python3.9/site-packages/dask_jobqueue/core.py:19: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/glade/u/home/mgrover/.local/lib/python3.9/site-packages/dask_jobqueue/core.py:19: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile
/glade/u/home/mgrover/.local/lib/python3.9/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use da

## Spin up a Dask Cluster

In [2]:
cluster = NCARCluster()
cluster.scale(40)
client = Client(cluster)
client

/glade/work/mgrover/miniconda3/envs/esm-collections-dev/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44795 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/44795/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/44795/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.57:41369,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/44795/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Setup a Cache Store

In [3]:
store = CacheStore(f'{tempfile.gettempdir()}/marbl-example-cache')
store

CacheStore(path='/glade/scratch/mgrover/marbl-example-cache', readonly=False, on_duplicate_key=<DuplicateKeyEnum.skip: 'skip'>, storage_options={})

## Epoch Mean Global Map Example
As mentioned before, in this first example, we will generate an analysis where we average over some `time` dimension, and plot a global map of some field(s)

In [14]:
def epoch_mean(ds):
    """Take the average over some time dimension"""
    return ds.mean(dim='time')

def convert_to_collection(keys, dsets):
    """Converts keys and datasets to an xcollection.Collection"""
    return xc.Collection({keys: dsets for keys, dsets in zip(keys, dsets)})

## Setup Tasks

In [15]:
@task
def read_catalog(path, multivar_row=False):
    """
    Reads in an intake-esm catalog from a given path

    Parameters
    ----------
    path: str, path to the associated intake-esm catalog json

    multivar_row: boolean, default=False, whether multiple variables are in each row

    Returns
    -------
    intake_esm.Catalog
    """
    if multivar_row:
        read_csv_kwargs = {"converters": {"variables": ast.literal_eval}}
    else:
        read_csv_kwargs = None

    return intake.open_esm_datastore(path, read_csv_kwargs=read_csv_kwargs)


@task
def subset_catalog(intake_esm_catalog, search_dict):
    """
    Subsets an intake-esm catalog given a search dictionary

    Parameters
    ----------
    intake_esm_catalog: intake_esm.Catalog

    search_dict: dict, dictionary to use for searching the catalog

    Returns
    -------
    catalog_subset: intake_esm.Catalog, catalog subsetted for given search
    """
    return intake_esm_catalog.search(**search_dict)


@task
def load_catalog(intake_esm_catalog, cdf_kwargs={}):
    """
    Loads the intake-esm catalog datasets into an xcollection.Collection

    Parameters
    ----------
    intake_esm_catalog: intake_esm.Catalog, catalog to read the datasets from

    cdf_kwargs: dict, dataset kwargs for intake-esm (cdf_kwargs)

    Returns
    -------
    collection: xcollection.Collection
    """
    return intake_esm_catalog.to_collection(cdf_kwargs=cdf_kwargs)


@task
def get_task_attr(obj, attr):
    """
    Retrives an attribute from a Prefect.Task

    Parameters
    ----------
    obj: Prefect.Task, the task you would like to retrieve the attributes from

    attr: str, attribute you would like to retrieve

    Returns
    -------
    object_out: Desired attribute
    """
    return list(getattr(obj, attr)())


@task(
    target="{task_name}-{key}.zarr",
    result=XpersistResult(store, serializer="xarray.zarr", serializer_dump_kwargs={"mode": "w"}),
)
def epoch_average(ds, key):
    return epoch_mean(ds)

## Setup the Flow

In [16]:
with Flow('epoch_mean') as epoch_mean_flow:
    collection_path = Parameter(
        'collection_path',
        default='/glade/work/mgrover/cesm2_le_test_data/catalogs/cesm2-le-subset.json',
    )
    multivar_row = Parameter('multi_var_row', default=False)
    search_dict = Parameter('search_dict', default={})
    cdf_kwargs = Parameter('cdf_kwargs', default=None)

    # Load the intake-esm catalog into memory
    data_catalog = read_catalog(collection_path, multivar_row)

    # Subset the catalog
    data_catalog_subset = subset_catalog(data_catalog, search_dict)

    # Convert to catalog to an xcollection
    collection = load_catalog(data_catalog_subset, cdf_kwargs=cdf_kwargs)

    values = get_task_attr(collection, "values")
    keys = get_task_attr(collection, "keys")

    # Apply the epoch average operator
    # Build a prefect flow wrapper - runs flow, returns collection
    epoch_average_collection = epoch_average.map(values, keys)

## Contain the flow within a function

In [17]:
def epoch_mean_flow_collection(collection_path, multi_var_row=False, search_dict={}, cdf_kwargs=None):
    """
    Parameters
    ----------
    collection_path = str, path to intake-esm json
    multi_var_row = bool, True for history files, False for timeseries files
    search_dict = dict, search dictionary
    cdf_kwargs = dict, open dataset arguments
    """
    run_flow = epoch_mean_flow.run(collection_path=collection_path,
                                   multi_var_row=multi_var_row,
                                   search_dict=search_dict,
                                   cdf_kwargs=cdf_kwargs)
    
    return convert_to_collection(run_flow.result[keys].result,
                                 run_flow.result[epoch_average_collection].result)
    

In [18]:
epoch_mean_collection = epoch_mean_flow_collection(collection_path='/glade/work/mgrover/cesm2_le_test_data/catalogs/cesm2-le-subset.json',
                                                   search_dict={'stream': 'pop.h', 'variable': 'PO4'}, cdf_kwargs={})

[2022-01-12 13:58:12-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'epoch_mean'
[2022-01-12 13:58:12-0700] INFO - prefect.TaskRunner | Task 'multi_var_row': Starting task run...
[2022-01-12 13:58:12-0700] INFO - prefect.TaskRunner | Task 'multi_var_row': Finished task run for task with final state: 'Success'
[2022-01-12 13:58:12-0700] INFO - prefect.TaskRunner | Task 'collection_path': Starting task run...
[2022-01-12 13:58:12-0700] INFO - prefect.TaskRunner | Task 'collection_path': Finished task run for task with final state: 'Success'
[2022-01-12 13:58:12-0700] INFO - prefect.TaskRunner | Task 'search_dict': Starting task run...
[2022-01-12 13:58:12-0700] INFO - prefect.TaskRunner | Task 'search_dict': Finished task run for task with final state: 'Success'
[2022-01-12 13:58:12-0700] INFO - prefect.TaskRunner | Task 'cdf_kwargs': Starting task run...
[2022-01-12 13:58:12-0700] INFO - prefect.TaskRunner | Task 'cdf_kwargs': Finished task run for task with final state: 'Succ

[2022-01-12 13:58:22-0700] INFO - prefect.TaskRunner | Task 'load_catalog': Finished task run for task with final state: 'Success'
[2022-01-12 13:58:22-0700] INFO - prefect.TaskRunner | Task 'get_task_attr': Starting task run...
[2022-01-12 13:58:22-0700] INFO - prefect.TaskRunner | Task 'get_task_attr': Finished task run for task with final state: 'Success'
[2022-01-12 13:58:22-0700] INFO - prefect.TaskRunner | Task 'get_task_attr': Starting task run...
[2022-01-12 13:58:22-0700] INFO - prefect.TaskRunner | Task 'get_task_attr': Finished task run for task with final state: 'Success'
[2022-01-12 13:58:22-0700] INFO - prefect.TaskRunner | Task 'epoch_average': Starting task run...
[2022-01-12 13:58:22-0700] INFO - prefect.TaskRunner | Task 'epoch_average': Finished task run for task with final state: 'Mapped'
[2022-01-12 13:58:22-0700] INFO - prefect.TaskRunner | Task 'epoch_average[0]': Starting task run...
[2022-01-12 13:58:22-0700] INFO - prefect.TaskRunner | Task 'epoch_average[0]':

In [19]:
epoch_mean_collection